In [9]:
# !pip install -q tensorflow-gpu=2.0.0-rc1
import pandas as pd
import tensorflow as tf
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 下载数据

In [16]:
heart_file_path = tf.keras.utils.get_file(fname = "heart.csv", origin = "https://storage.googleapis.com/applied-dl/heart.csv")
!head {heart_file_path}

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0
56,1,2,120,236,0,0,178,0,0.8,1,0,normal,0
62,0,4,140,268,0,2,160,0,3.6,3,2,normal,1
57,0,4,120,354,0,0,163,1,0.6,1,0,normal,0
63,1,4,130,254,0,2,147,0,1.4,2,1,reversible,1


## 读取数据

In [24]:
df = pd.read_csv(heart_file_path)
df.head()
df.dtypes

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

In [27]:
df["thal"] = pd.Categorical(df["thal"])
df["thal"] = df.thal.cat.codes
df.head()
df.values

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3


array([[63.,  1.,  1., ...,  3.,  0.,  2.],
       [67.,  1.,  4., ...,  2.,  3.,  3.],
       [67.,  1.,  4., ...,  2.,  2.,  4.],
       ...,
       [65.,  1.,  4., ...,  2.,  1.,  4.],
       [48.,  1.,  4., ...,  1.,  2.,  4.],
       [63.,  0.,  4., ...,  2.,  3.,  4.]])

## 构建 TensorFlow Dataset

In [26]:
target = df.pop("target")
target.head()
target.values

0    0
1    1
2    0
3    0
4    0
Name: target, dtype: int64

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,

In [22]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
for feature, target in dataset.take(5):
    print(f"Feature: {feature}, Target: {target}")

Feature: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Feature: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Feature: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Feature: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Feature: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


# 1.构建训练数据、模型

## 1.1 构建训练数据

In [29]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## 1.2 构建模型

In [30]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

model = get_compiled_model()
model.fit(train_dataset, epochs = 15)

Epoch 1/15
303/303 [==============================] - 1s 571us/step - loss: 2.6222 - accuracy: 0.7099
Epoch 2/15
303/303 [==============================] - 0s 582us/step - loss: 1.1615 - accuracy: 0.6957
Epoch 3/15
303/303 [==============================] - 0s 607us/step - loss: 1.0760 - accuracy: 0.7190
Epoch 4/15
303/303 [==============================] - 0s 596us/step - loss: 0.8303 - accuracy: 0.7780
Epoch 5/15
303/303 [==============================] - 0s 615us/step - loss: 0.7715 - accuracy: 0.7822
Epoch 6/15
303/303 [==============================] - 0s 599us/step - loss: 0.9911 - accuracy: 0.7373
Epoch 7/15
303/303 [==============================] - 0s 586us/step - loss: 0.9035 - accuracy: 0.7087
Epoch 8/15
303/303 [==============================] - 0s 584us/step - loss: 0.8479 - accuracy: 0.7366
Epoch 9/15
303/303 [==============================] - 0s 582us/step - loss: 0.7759 - accuracy: 0.7257
Epoch 10/15
303/303 [==============================] - 0s 623us/step - loss: 0.642

# 2.构建训练数据、模型

In [38]:
inputs = {
    key: tf.keras.layers.Input(shape = (), name = key) for key in df.keys()
}
x = tf.stack(list(inputs.values()), axis = -1)
x = tf.keras.layers.Dense(10, activation = 'relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_func = tf.keras.Model(inputs = inputs, outputs = output)
model_func.compile(
    optimizer = 'adam',
   loss = 'binary_crossentropy',
   metrics = ['accuracy']
)
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)
model_func.fit(dict_slices, epochs = 15)

Epoch 1/15
19/19 [==============================] - 0s 1ms/step - loss: 9.2713 - accuracy: 0.7493
Epoch 2/15
19/19 [==============================] - 0s 1ms/step - loss: 5.2372 - accuracy: 0.7493
Epoch 3/15
19/19 [==============================] - 0s 1ms/step - loss: 1.8918 - accuracy: 0.7377
Epoch 4/15
19/19 [==============================] - 0s 1ms/step - loss: 0.9619 - accuracy: 0.7064
Epoch 5/15
19/19 [==============================] - 0s 1ms/step - loss: 0.8488 - accuracy: 0.7328
Epoch 6/15
19/19 [==============================] - 0s 1ms/step - loss: 0.8332 - accuracy: 0.7598
Epoch 7/15
19/19 [==============================] - 0s 1ms/step - loss: 0.8026 - accuracy: 0.7622
Epoch 8/15
19/19 [==============================] - 0s 1ms/step - loss: 0.7860 - accuracy: 0.7683
Epoch 9/15
19/19 [==============================] - 0s 1ms/step - loss: 0.7676 - accuracy: 0.7654
Epoch 10/15
19/19 [==============================] - 0s 1ms/step - loss: 0.7496 - accuracy: 0.7639
Epoch 11/15
19/19 [